In [84]:
import pandas as pd

urls_frame = pd.read_csv('datasets/data.csv')
num_examples = len(urls_frame)
chars = {}
hi = 0
for url in urls_frame['url']:
    for c in url:
        if c not in chars:
            chars[c] = 0
        chars[c] += 1

In [81]:
# we will use 1-hot vectors for the top 100 chars
char_f = sorted([(c, chars[c]) for c in chars], key=lambda v: -v[1])
char_count = 100
indices = {}
for i in range(char_count-1):
    indices[char_f[i][0]] = i
def char2index(c):
    if c in indices:
        return indices[c]
    else:
        return 99

print(char2index('o'))
print(char2index('¢'))

1
99


In [82]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

use_cuda = torch.cuda.is_available()


num_train = int(.9 * num_samples)
num_val = int(.1 * num_samples)

In [23]:
class ChunkSampler(sampler.Sampler):
    def __init__(self, num_samples, start=0):
        self.num_samples = num_samples
        self.start = start
    
    def __iter__(self):
        return iter(range(self.start, self.start+self.num_samples))
    
    def __len__(self):
        return self.num_samples


class UrlDataset(Dataset):
    
    def __init__(self, csv_file, start, end):
        self.urls_frame = pd.read_csv(csv_file)
        self.batch_size = 16
    
    def __len__(self):
        return len(self.urls_frame)
    
    def __getitem__(self, idx):
        return self.urls_frame[idx]

batch_size = 16
loader_train = DataLoader(UrlDataset('datasets/data.csv'), sampler=ChunkSampler())

In [ ]:
# takes input urls, returns Pr{malicious}

class CharRNN(nn.Module):
    
    def __init__(self, hidden_size):
        self.layer1 = 
    
    def forward(self, input):
        
    

In [ ]:
def check_accuracy(model, loader):
    num_correct = 0
    num_samples = 0
    model.eval()
    for X, y in loader:
        X_var = Variable(X, volatile=True)

        scores = model(X_var)
        preds = scores.data.cpu() > 0.5

        num_correct += (preds == y).sum()
        num_samples += preds.size(0)

    acc = float(num_correct) / num_samples
    print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    

def train(model, loader_train, criterion, optimizer):
    model.train()
    for t, (X, y) in enumerate(loader_train):
        X_var = Variable(X)
        y_var = Variable(y).long()

        scores = model(X_var)

        loss = criterion(scores, y_var)
        if (t+1) % args.print_every == 0:
            print('t = %d, loss = %.4f' % (t+1, loss.data[0]))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
model = CharConv1d(5)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
num_epochs = 5
for epoch in range(num_epochs):
    check_accuracy(model, loader_val)
    print('Starting epoch %d / %d' % (epoch+1, num_epochs))
    train(model, loader_train, criterion, optimizer)